# IVR7 Intermediate Example
This example aims to highlight the follow:
* using time, distance and a single capacity dimension
* Illustrates using time, distance and a single capacity dimension
* Location-windows (08:00 -> 14:00)
* Pickup-dropoff tasks (with task-times)
* One vehicle class (same travel profile)
* Two vehicle-cost classes
* Multiple vehicles (2xc1, 3xc2)
* Heterogeneous fleet (2 ton and 3 ton capacity)
* Lunch breaks (1 hour break around 12:00)


## Requirements
This example assumes you've configured an api-key with the required services enabled (see the [portal](portal.icepack.ai) for configuration details) and that you're familiar with loading and working with protobuf models.

## Data
We're going to load sample data which has order sizes and durations in a tabular format.


In [ ]:
import pandas
df = pandas.read_csv('../sample_data/publist_orders.csv')
print(df.head())

## Model Configuration
This model builds on the ivr7-basic example.
The same dimensions (distance,time and capacity) are used:

In [ ]:
exec(open('apiHelper.py').read()) # import some api-helper classes which we've written for you.
exec(open('ivr7-model-helper.py').read()) # import some modelling helpers


api = apiHelper(modelType="ivr7-kt461v8eoaif") # set the model type to the ivr7 model

sr = ivr7_kt461v8eoaif_pb2.SolveRequest()
sr.solveType = 0 # optimise solve request

m = sr.model

m.dimensions.CopyFrom(make_distance_time_cap_dims())

print(m)


## Locations

We're going to treat the first point as where vehicles are going to begin and end each day. Unlike the tsp/cvrp/pdp models, the ivr7 requires that you specify the unique locations that are going to be used in the model as a separate entity. The reason for this is that you can then specify the locations once, and reference those locations by id for other entities (such and vehicles/jobs/tasks).

We're going to add arrival windows to the locations (08:00 - 14:00). We can do this by first building the locations (as before) and then adding a window attribute to each location. This constraint means that we're required to arrive between the specified start and end.

In [ ]:
del m.locations[:] # just in case you're running this twice, clear the field before extending it.
m.locations.extend(make_locations(df)) #using the wrapper function



la = ivr7_kt461v8eoaif_pb2.Location.Attribute()
la.dimensionId = 'time'
la.quantity = 0 # if you have a fixed time for entering a location you could place it here
w = ivr7_kt461v8eoaif_pb2.Window()
w.start = 8*60
w.end = 14*60
la.arrivalWindows.append(w)

for i in range(0, len(m.locations)):
    m.locations[i].attributes.append(la)

print(str(len(m.locations)) + " locations\n")
# check the first locations' configuration
print(m.locations[0])

## Jobs
For each job, there is a unique ID, a penalty to be applied if the job is not completed as well as associated tasks.

Jobs with higher penalties will be prioritised over jobs with lower penalties.

### Tasks
Jobs also have associated tasks with each task occuring at assigned locations and must be completed by the same vehicle. If one task cannot be completed then the entire job will unperformed in the solution. If a job is unperformed, it will be returned in the Infeasibilites message of the solution along with reasons as to why it could not be completed.

We've constructed some jobs with pickups and dropoffs, loading and offload times, as well as the contribution to the capacity dimension (through the attributes). In this example, all pickup up orders originate at the guiness storehouse and deliver to each customer in the list. `make_job_time_cap` is just a simple function to create this particular style of request, but you can make your own.
 
### Dimension - Capacity
A positive quantity value indicates goods being added to the vehicle and a negative quantity value indicates goods being removed from the vehicle.


In [ ]:
njobs = (df.shape[0])
src = [0] * (njobs-1)
dest =  range(1, njobs)

del m.jobs[:]
m.jobs.extend(make_job_time_cap(df, src, dest))
# lets inspect just the first job created
print(m.jobs[0])

### Vehicle Cost Classes
In this example we're going to create two vehicle cost classes. One which is cheaper on marginal costs (distance and time), but more expensive in fixed cost. Similarly, the second class is cheaper in fixed cost, but more expensive on marginal (travel and time) costs.

In [ ]:
del m.vehicleCostClasses[:]
m.vehicleCostClasses.append(make_vcc_simple('vcc1', 1000, 1.001e-2, 1.001e-2, 1.001e-2, 1, 3))
m.vehicleCostClasses.append(make_vcc_simple('vcc2', 1200, 1.001e-1, 1.001e-1, 1.001e-1, 0.6, 2.5))
print(m.vehicleCostClasses)

### Vehicle Class
A vehicle class describes how the vehicle moves through the network. In other words, we can use the standard network travel speeds, or we could make the vehicle move slower/faster relative to the road network. We could also attach transit rules here which are great for modelling lunch breaks, refueling stops etc. (i.e. conditional triggers on the cumul values of the dimension). Transit rules are covered in advanced examples.
In this example, we're still going to create one "movement" class:


In [ ]:
del m.vehicleClasses[:]
m.vehicleClasses.append(make_vc_simple('vc1', 1, 1, 1, 1))
print("Vehicle Classes")
print(m.vehicleClasses)

### Vehicles
The solver requires a fixed number of vehicle instances to work with. In this model, we can create 5 vehicles all starting and ending their day at the depot. Our 5 vehicels are going to consist of:
* 2 x two-ton vehicles  (capacity limit 2000)
* 3 x three-ton vehicles (capacity limit 3000)

We're doing this in code here to minimise the sample data required, but you can link this up to another table or input data stream as needed.

In [ ]:
del m.vehicles[:]
for i in range(1,5):
    vcc = 'vcc1'
    vcap = 2000
    if i >= 3:
        vcap = 3000
        vcc = 'vcc2'
      
    m.vehicles.append( make_vehicle_cap('vehicle_' + str(i), 'vc1', vcc,
                                        vcap, # the vehicle capacity
                                       df['id'][0], # the start location
                                       df['id'][0], # the end location
                                       7*60,        # the start time (7AM)
                                       18*60))       # the end time  (6PM)
print("Vehicle 1")
print(m.vehicles[0])
print("Vehicle 3")
print(m.vehicles[2]) # note the capacities are different for these two vehicles.

### Sending the model
We now have a complete model specified so we can submit it to the api.

In [ ]:
reqId = api.Post(sr)
print(reqId) # if you want to see what the guid looks like.

### Retrieve the model response
We can use the api-helper to grab the solution response using the request-id provided by the post.

In [ ]:
sol = api.Get(reqId)

print(sol.objective) # This is the cost of the solution returned by the api.


### Tabluating the response

It's quite convenient to table the response data from the api into a collection of stops (nodes) and inter-stops (edges). The reason for this separation between nodes and edges is that they represent different states in the model. Node dimensional quantities refer to quantities incurred while _standing_ and, by contrast, when talking about edges the quantities refer to the _movement_ between tasks. 


In [ ]:
print(sol.routes[0])

In [ ]:
t = tabulate(sr, sol) # pass in the solve request (we grab the locations from the mode) and the solution response
print(t.keys())
print("node table has " + str(t['nodes'].shape[0]) + ' rows and ' + str(t['nodes'].shape[1]) + ' colums' )
print("edge table has " + str(t['edges'].shape[0]) + ' rows and ' + str(t['edges'].shape[1]) + ' colums' )

### Inspecting the tabulated result

Lets start with the stop-table (or node data):


In [ ]:
t['nodes']


Similarly for the edges, we can check the table. Note that the geometries for the edges have been compressed into a list object within each row  (this makes things a touch easier for plotting). 



In [ ]:
t['edges']


### Visualising the response

Lets go ahead and visualise the routes.


In [ ]:
from ipyleaflet import Map, Polyline, Circle, LayerGroup

cvec = list(['blue', 'red', 'orange', 'green', 'black'])
cdict = {}
for i in range(0,5):
    cdict['vehicle_' + str(i)] = cvec[i]


# form some locations to add to the map (just green dots seems ok)
locs = list()
for index, p in t['nodes'].iterrows():
    circle = Circle()
    circle.location = (p['y'], p['x'])
    circle.radius = 10
    circle.color = "green"
    locs.append(circle)

center = [df['Y'][1],df['X'][1]] # just use the first point as the center (i.e. the depot)
m = Map(scroll_wheel_zoom=True, center=center, zoom=12)
for i, gs in enumerate(t['edges']['geometry']):
    m.add_layer(Polyline(locations=gs,color=cdict[t['edges']['vehicleId'][i]],fill=False))

m.add_layer(LayerGroup(layers=(locs)))
m

### Transit Rules
We can create a simple example which uses a lunch break. We have a lot of control over how a lunch-break is executed in the solver. For example,  we can specify where the lunchbreak may occur between stops, at stops, or either. We recommend a read through the specification in the [documentation](https://docs.scrudu.io/ivr/transit-rule/). In this example, we're going to create a lunch break rule which specifies a 60 minute break needs to be taken in the first spot available after 12:00.

In [ ]:
m = sr.model # we accidentally killed this object above when we created the map :-) just link to it again!
del m.transitRules[:]
m.transitRules.append(make_lunch_break_rule('lunch_break_rule', 'lunchy_munchy_', 12*60, 60))

m.vehicleClasses[0].transitRuleIds.append('lunch_break_rule') # assign the transit rule to the vehicle-class

print(m.transitRules) 
# the rule is "trigger" based. So when we pass a certain "value" apply the following "quantity"

### Rerun the model and grab the results


In [ ]:
reqId = api.Post(sr)
sol = api.Get(reqId)
print(sol.objective) # This is the cost of the solution returned by the api.
t = tabulate(sr, sol) # pass in the solve request (we grab the locations from the mode) and the solution response


### Recovering the transit rules
The result returned by the solver includes a special section called `transitRuleAttributes` for each route which details where the points are wher ea vehicle exeutes a transit rule. In the example below, we can see that a 60 minute interval is applied to the time dimension at the first time junction which spans 720 minutes (i.e. 12:00) 


In [ ]:
for _, e in enumerate(sol.routes):
    if len(e.transitRuleAttributes) > 0:
        print(e.transitRuleAttributes)


### Tabulated transit rules
We can also pull out the tabulated transit rules from the helper function. Makes things a touch easier to read:

In [ ]:
t['transitrules']

### What next?
If this example makes sense you can probably proceed to working with more complex modelling activities. See the IVR7 intermediate2 and advanced examples.